# Recalculate anchors

In [10]:
heatmap = warped_heatmaps_new.repeat(2, 1, 1, 1)[:,1:13]
heatmap.shape

NameError: name 'warped_heatmaps_new' is not defined

In [11]:
pred_orig.core.shape

torch.Size([1, 1, 3, 3, 1])

In [123]:
minxs.unsqueeze(-1)

tensor([[129],
        [140],
        [130],
        [142],
        [129],
        [136],
        [130],
        [140],
        [112],
        [147],
        [114],
        [137]], device='cuda:0')

In [124]:
t = torch.tensor([[[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]], 
                  [[10, 11, 12],
                   [13, 14, 15],
                   [16, 17, 18],
                  ]])

t_ = t.view(-1, 9)
print(t_)

# row number 2, col 1 --> 2 * 3 + col
torch.gather(t_, dim = 1, index = torch.tensor([[7],[2]]))

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18]])


tensor([[ 8],
        [12]])

In [125]:
torch.tensor([True, False]) | torch.tensor([True, False])

tensor([ True, False])

In [174]:
#mask = heatmap == 0
#(heatmap*mask).sum(dim=0)/mask.view(-1, 256 * 256).sum(dim=1)
mean = heatmap.view(-1, 256 * 256).sum(dim=1) / (heatmap != 0).view(-1, 256 * 256).sum(dim=1)
mean.min()
heatmap[heatmap > heatmap[heatmap != 0].mean()].mean()
heatmap[[2, 3, 4]].shape

torch.Size([3, 256, 256])

In [162]:
torch.tensor([torch.nan, 2]).min()

tensor(nan)

In [2]:
heatmap = warped_heatmaps_new.repeat(2, 1, 1, 1)[:,1:13]
#heatmap = warped_heatmaps_new.repeat(2)
#print(heatmpa)
double = pred_orig.double

for i in range(warped_heatmaps_new.shape[0]): # for each in batch
    heatmap = heatmaps[i]
    #print(heatmap.shape)
    threshold = heatmap[heatmap >= heatmap[heatmap != 0].mean()].mean()
    where = heatmap > threshold  # a high threshold gets the anchors closer to the parts, empirically, 
    #print(torch.nonzero(torch.sum(where, 1)))
    xsums = torch.nonzero(torch.sum(where, 1))
    #print(torch.sum(where, 1).shape)
    bincount = torch.bincount(xsums[:,0])
    unique = torch.unique(xsums[:,0])
    #print(xsums)
    indices = torch.cumsum(torch.cat([torch.tensor([0], device = device), bincount], dim = 0), dim=0)
    minxs = xsums[indices[:-1],:][:,1]
    maxxs = xsums[(indices - 1)[1:],:][:,1]

    xdist = (maxxs - minxs)
    
    #print(xdist.shape)
    
    ysums = torch.nonzero(torch.sum(where, 2))
    bincount = torch.bincount(ysums[:,0])
    #print(bincount)
    indices = torch.cumsum(torch.cat([torch.tensor([0], device = device), bincount], dim = 0), dim=0)
    #print(indices.shape)
    minys = ysums[indices[:-1],:][:,1]
    maxys = ysums[(indices - 1)[1:],:][:,1]

    ydist = (maxys - minys)
    
    isvertical = ydist > xdist
    
    heatmap_ = heatmap.view(-1,256 * 256) 

    # row number 2, col 1 --> 2 * 3 + col
    
    # top left to bottom right
    minyminx= torch.gather(heatmap_, dim = 1, index = (minys * 256 + minxs).unsqueeze(-1)).squeeze(1)
    
    # bottom left to top right
    minymaxx= torch.gather(heatmap_, dim = 1, index = (minys * 256 + maxxs).unsqueeze(-1)).squeeze(1)
      
    centerxs = (minxs + maxxs) // 2
    centerys = (minys + maxys) // 2
    
    # horizontal
    horizontal= torch.gather(heatmap_, dim = 1, index = (centerys * 256 + minxs).unsqueeze(-1)).squeeze(1)
    
    # vertical
    vertical = torch.gather(heatmap_, dim = 1, index = (minys * 256 + centerxs).unsqueeze(-1)).squeeze(1)
    
    straight = isvertical * vertical + (~isvertical * horizontal)

    topleft = (minyminx > minymaxx)
    
    isstraight = ((straight > minyminx) | ((straight > minymaxx)))

    isvertical = isvertical * isstraight
    topleft = topleft * ~isstraight
    bottomleft = ~isstraight * ~topleft
    ishorizontal = isstraight * ~isvertical
    
    
    x0 = ((isvertical * centerxs) + (ishorizontal * minxs) + (topleft * minxs) + (bottomleft * maxxs)).unsqueeze(-1)
    y0 = ((isvertical * minys) + (ishorizontal * centerys) + (topleft * minys) + (bottomleft * minys)).unsqueeze(-1)
    
    x1 = ((isvertical * centerxs) + (ishorizontal * maxxs) + (topleft * maxxs) + (bottomleft * minxs)).unsqueeze(-1)
    y1 = ((isvertical * maxys) + (ishorizontal * centerys) + (topleft * maxys) + (bottomleft * maxys)).unsqueeze(-1)
    
    #print(x0.shape)
    double[i, unique, 0, 0] = x0.float()
    double[i, unique, 0, 1] = y0.float()
    double[i, unique, 1, 0] = x1.float()
    double[i, unique, 1, 1] = y1.float()
    eucs = torch.cdist(pred_new.double[0,unique,0,0:2].unsqueeze(1).squeeze(3), double[i, unique, :, 0:2].squeeze(3))
    change = eucs[:,0,0] > eucs[:,0,1]
    double[i,change,0,0] = x1[change].float()
    double[i,change,0,1] = y1[change].float()
    double[i,change,1,0] = x0[change].float()
    double[i,change,1,1] = y0[change].float()
    #print(eucs)
    #print(change)
    #print(pred_new.double[0,:,0,0:2].unsqueeze(1).squeeze(3).shape) # first joint
    #print(double[i, :, :, 0:2].squeeze(3).shape)
    #print(torch.cdist(pred_new.double[0,:,0,0:2].unsqueeze(1).squeeze(3), double[i, :, :, 0:2].squeeze(3)))
    

NameError: name 'warped_heatmaps_new' is not defined

# Adjust anchor loss

In [125]:
from utils.helper import load_config
cfg = load_config("../../../configs/direct_loss_calc_anchors.yaml")

In [126]:
import sys
sys.path.append("..")

import torch.nn as nn
import torch

l2_loss = nn.MSELoss()

l1_loss = nn.L1Loss()

import torch

device = torch.device('cuda:0')

from utils.helper import load_anchor_points_doubles_only

double = load_anchor_points_doubles_only("../../../" + cfg['anchor_pts_path'], device, 1)



In [127]:
size = 256

In [128]:
double.shape

torch.Size([1, 14, 2, 3, 1])

In [129]:
loss = 0
# normalize to range 0, 1
double = double/size # core -- hip, thigh, shin, shoulder, arm, forearm -- head
# batch_size, which part, which anchor

# indices
core = 0 
left_hip = 1
right_hip = 2
left_thigh = 3
right_thigh = 4
left_shin = 5
right_shin = 6
left_shoulder = 7
right_shoulder = 8
left_arm = 9
right_arm = 10
left_forearm = 11
right_forearm = 12
head = 13

pairs = ((head, core), (core, left_shoulder), (left_shoulder, left_arm), (left_arm, left_forearm), 
        (core, right_shoulder), (right_shoulder, right_arm), (right_arm, right_forearm),
        (core, left_hip), (left_hip, left_thigh), (left_thigh, left_shin), 
        (core, right_hip), (right_hip, right_thigh), (right_thigh, right_shin))

for pair in pairs:
    # for each pair, find the connecting anchor (among 4 distance)
    # head[0] - core[0], head[0] - core[1], head[1] - core[0], head[1] - core[0]
    # head and core
    indices1 = double[:, pair[0]] # shape [1, 2, 3, 1]
    indices2 = double[:, pair[1]]
    losses = torch.stack([l2_loss(indices1[:,0], indices2[:,0]), l2_loss(indices1[:,1], indices2[:,1]), l2_loss(indices1[:,1], indices2[:,0]), l2_loss(indices1[:,0], indices2[:,1])])

    loss += torch.min(losses)


In [130]:
threshold = nn.Threshold(1, 0)

double = double.view(double.shape[0], -1, double.shape[3])

# normalize to range -1  to 1
double = (double / size) * 2 - 1
double.shape


torch.Size([1, 28, 3])

In [135]:
comb = torch.cat([double, double], dim=1)
comb.shape

torch.Size([1, 56, 3])

In [133]:
threshold(torch.abs(double)).sum(1)

tensor([[0., 0., 0.]], device='cuda:0')